In [1]:
import numpy as np
import os
import torch
from sklearn.externals import joblib

from pytorch_utils.datasets import ArrayDataset
from pytorch_utils.models import SparseModel, SparseModelEmbed

In [2]:
data_path = 'data/'
features_path = os.path.join(data_path, 'features', str(0))
label_path = os.path.join(data_path, 'labels')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
features_dict = joblib.load(os.path.join(features_path, 'features.pkl'))
label_dict = joblib.load(os.path.join(label_path, 'label_dict.pkl'))

In [5]:
outcome = 'mortality'

In [6]:
data_dict = {split: features_dict[split]['features'] for split in features_dict.keys()}
outcome_dict = {split : label_dict[split][outcome] for split in label_dict.keys()}

In [7]:
config_dict = {
    'input_dim' : data_dict['train'].shape[1],
    'output_dim' : 2,
    'lr' : 1e-3,
    'num_epochs' : 5,
    'batch_size' : 256,
    'iters_per_epoch' : None,
}

In [8]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [9]:
%%time
model = SparseModelEmbed(config_dict)
result = model.train(data_dict, outcome_dict)
model.predict(data_dict, outcome_dict, keys = ['test'])

Epoch 0/4
----------
Phase: train:
 loss: 0.100647,
 auc: 0.794014, auprc: 0.126140, brier: 0.022207,
Phase: val:
 loss: 0.085385,
 auc: 0.857317, auprc: 0.176937, brier: 0.020041,
Best model updated
Epoch 1/4
----------
Phase: train:
 loss: 0.058739,
 auc: 0.954248, auprc: 0.517860, brier: 0.015067,
Phase: val:
 loss: 0.084638,
 auc: 0.855236, auprc: 0.199238, brier: 0.019530,
Best model updated
Epoch 2/4
----------
Phase: train:
 loss: 0.045392,
 auc: 0.977849, auprc: 0.701544, brier: 0.011721,
Phase: val:
 loss: 0.086227,
 auc: 0.852627, auprc: 0.209251, brier: 0.019413,
Epoch 3/4
----------
Phase: train:
 loss: 0.037646,
 auc: 0.986539, auprc: 0.794104, brier: 0.009681,
Phase: val:
 loss: 0.089232,
 auc: 0.845028, auprc: 0.211528, brier: 0.019411,
Epoch 4/4
----------
Phase: train:
 loss: 0.032327,
 auc: 0.990809, auprc: 0.849499, brier: 0.008240,
Phase: val:
 loss: 0.093078,
 auc: 0.836366, auprc: 0.211545, brier: 0.019444,
Best val performance: 0.084638
CPU times: user 1min 30s, 

In [10]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
%%time
model = SparseModel(config_dict)
result = model.train(data_dict, outcome_dict)
model.predict(data_dict, outcome_dict, keys = ['test'])

Epoch 0/4
----------
Phase: train:
 loss: 0.518347,
 auc: 0.557648, auprc: 0.028824, brier: 0.071346,
Phase: val:
 loss: 0.324669,
 auc: 0.640300, auprc: 0.051642, brier: 0.036902,
Best model updated
Epoch 1/4
----------
Phase: train:
 loss: 0.264732,
 auc: 0.712042, auprc: 0.086046, brier: 0.034242,
Phase: val:
 loss: 0.280925,
 auc: 0.687636, auprc: 0.069175, brier: 0.032661,
Best model updated
Epoch 2/4
----------
Phase: train:
 loss: 0.203207,
 auc: 0.777397, auprc: 0.145960, brier: 0.029281,
Phase: val:
 loss: 0.258564,
 auc: 0.709037, auprc: 0.078001, brier: 0.031472,
Best model updated
Epoch 3/4
----------
Phase: train:
 loss: 0.159498,
 auc: 0.825505, auprc: 0.221496, brier: 0.025525,
Phase: val:
 loss: 0.242524,
 auc: 0.724144, auprc: 0.083796, brier: 0.031647,
Best model updated
Epoch 4/4
----------
Phase: train:
 loss: 0.126198,
 auc: 0.863853, auprc: 0.309469, brier: 0.022100,
Phase: val:
 loss: 0.232096,
 auc: 0.734766, auprc: 0.087198, brier: 0.031988,
Best model updated


In [12]:
## Test difference in outputs:
model1 = SparseModelEmbed(config_dict)
model2 = SparseModel(config_dict)

In [13]:
state_dict = model1.model.state_dict()
state_dict['weight'] = state_dict['weight.weight']
del state_dict['weight.weight']
model2.model.load_state_dict(state_dict)

In [14]:
print(model1.model.state_dict())
print(model2.model.state_dict())

OrderedDict([('bias', tensor([-0.6653,  0.4984], device='cuda:0')), ('weight.weight', tensor([[ 3.2055e-03, -3.4367e-04],
        [ 2.1737e-03, -9.4585e-04],
        [ 2.4648e-03, -1.8071e-03],
        ...,
        [-3.3004e-05, -3.2567e-03],
        [-1.3930e-03, -2.3600e-04],
        [-2.4835e-04, -3.8410e-03]], device='cuda:0'))])
OrderedDict([('weight', tensor([[ 3.2055e-03, -3.4367e-04],
        [ 2.1737e-03, -9.4585e-04],
        [ 2.4648e-03, -1.8071e-03],
        ...,
        [-3.3004e-05, -3.2567e-03],
        [-1.3930e-03, -2.3600e-04],
        [-2.4835e-04, -3.8410e-03]], device='cuda:0')), ('bias', tensor([-0.6653,  0.4984], device='cuda:0'))])


In [15]:
import copy
import torch.nn.functional as F

## Get some data
num_samples = 1000
inputs_model1 = data_dict['train'][:num_samples]
labels_model1 = torch.LongTensor(outcome_dict['train'][:num_samples]).to(device)

temp = ArrayDataset(inputs_model1)

inputs_model2 = temp.csr_to_tensor(copy.deepcopy(inputs_model1)).to(device)
labels_model2 = copy.deepcopy(labels_model1)

In [16]:
## Forward
outputs_model1 = model1.model.forward(inputs_model1)
outputs_model2 = model2.model.forward(inputs_model2)

In [17]:
loss1 = F.cross_entropy(outputs_model1, labels_model1)
loss2 = F.cross_entropy(outputs_model2, labels_model2)

In [18]:
loss1.backward()
loss2.backward()

In [19]:
print(np.allclose(outputs_model1.detach().cpu().numpy(), 
                  outputs_model2.detach().cpu().numpy()))
print(np.allclose(loss1.detach().cpu().numpy(), loss2.detach().cpu().numpy()))
print(np.allclose(model1.model.weight.weight.grad.detach().cpu().numpy(), 
                  model2.model.weight.grad.detach().cpu().numpy()))
print(np.allclose(model1.model.bias.grad.detach().cpu().numpy(), 
                  model2.model.bias.grad.detach().cpu().numpy()))

True
True
True
True
